In [ ]:
def js_to_python(s):
    d = {
        "true": "True",
        "false": "False",
        "null": "None",
        "[]": "None",
        "][": "],["
    }
    for k, v in d.items():
        s = s.replace(k, v)
    return s

def read_line(s):
    try:
        a, b = eval(js_to_python(s))
        return a
    except Exception as e:
        print(e)
        return eval(js_to_python(s))
        return []

def listdecorator(f):
    return lambda x: list(f(x))

@listdecorator
def get_trial_data(exp_dicts):
    return filter(is_trial, exp_dicts)
        
def is_trial(d):
    st = 'stimulus_type'
    return (d[st] == 'test') if (st in d.keys()) else False

def get_all_stimuli(trials):
    return [os.path.basename(trial[s]) for trial in trials for s in ['left', 'right']]

def get_group(fn):
    for g in ['low', 'high', 'medium']:
        if g in fn:
            return g

def nonempty_vp_idx(Ns, thr=0):
    return map(lambda x: x[0], (filter(lambda x: x[1] > thr, enumerate(Ns))))

In [ ]:
fn = '../data/experiment/jatos_results_20230119081857.txt'
with open(fn) as fh:
    raw_data = [read_line(s) for s in fh]
    #raw_data = fh.readlines()
len(raw_data)

In [ ]:
def get_responses(vp_data):
    return list(map(lambda d: (d['response'], d['bot_response']), filter(lambda d: 'bot_response' in d.keys(), vp_data)))

def result(t):
    return 0 if t[0] == t[1] else -1 if t[0]+t[1] in 'RPSR' else 1

In [ ]:
from collections import Counter
vp = 0
strats = ['nash_equilibrium_strategy', 'winstay', 'rotate_strategy', 'learn_preference', 'dont_always_copy_opponent_move']
# list(zip(get_responses(raw_data[vp]), map(result, get_responses(raw_data[vp]))))
for vp in range(len(raw_data)):
    strategy = strats[list(map(lambda d: d['strategy'], filter(lambda d: 'strategy' in d.keys(), raw_data[vp])))[0]]
    r = Counter(map(result, get_responses(raw_data[vp]))) 
    print(f'VP {vp} ({strategy}):', r[1] / (r[-1] + r[1]))